In [1]:
import pandas as pd
import requests

characters = pd.read_csv('vertex.csv')
characters = list(characters['Label'])

In [0]:
S = requests.Session()

def get_character_page(title):


  URL = "https://gameofthrones.fandom.com/api.php"

  PARAMS = {
      "action": "parse",
      "format": "json",
      "page": title,
      "prop": "wikitext",
      "formatversion": "2"
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()

  return DATA

character_contents = {}
for character in characters:
  character_contents[character] = get_character_page(character)

In [0]:
contents = {}
for title in character_contents:
  contents[title] = character_contents[title]['parse']['wikitext']['*']

In [0]:
contents_df = pd.DataFrame(contents.items(), columns=['wiki_title', 'wiki_content'])

In [44]:
contents_df

,wiki_title,wiki_content
0,Cassana Estermont,#REDIRECT[[Cassana Baratheon]]
1,Merchant captain,{{Character\n| Title = Merchant captain\n| Ima...
2,Fennesz,{{Character\n| Title=Fennesz\n| Image=Fennesz....
3,Musician 5 (The Door),{{DISPLAYTITLE:Musician 5 <small>(The Door)</s...
4,Valryon,{{Animal\n|Title = Valryon\n|Season = [[Season...
...,...,...
2041,Mordane,{{Character\n| Title=Mordane\n| Image=MordaneC...
2042,Great Master 5,{{Heraldry\n|image=SlaveMasters-Shield.PNG\n|l...
2043,Red Priestess (High Sparrow),{{DISPLAYTITLE:Red Priestess <small>(High Spar...
2044,Dareon,{{Heraldry\n|image=Night's-Watch-Main-Shield.P...


## Delete redirect links

In [61]:
not_redirected = contents_df[~contents_df.wiki_content.str.lower().str.contains('#redirect')]
not_redirected

,wiki_title,wiki_content
1,Merchant captain,{{Character\n| Title = Merchant captain\n| Ima...
2,Fennesz,{{Character\n| Title=Fennesz\n| Image=Fennesz....
3,Musician 5 (The Door),{{DISPLAYTITLE:Musician 5 <small>(The Door)</s...
4,Valryon,{{Animal\n|Title = Valryon\n|Season = [[Season...
5,Jaremy Mallister,{{Heraldry\n|image = House-Mallister-Main-Shie...
...,...,...
2041,Mordane,{{Character\n| Title=Mordane\n| Image=MordaneC...
2042,Great Master 5,{{Heraldry\n|image=SlaveMasters-Shield.PNG\n|l...
2043,Red Priestess (High Sparrow),{{DISPLAYTITLE:Red Priestess <small>(High Spar...
2044,Dareon,{{Heraldry\n|image=Night's-Watch-Main-Shield.P...


In [0]:
not_redirected.to_csv('not_redirected_titles.csv')


In [34]:
not_redirected = pd.read_csv('not_redirected_titles.csv')

## Filter pages without "{{Character" substring

In [50]:
only_characters = not_redirected[not_redirected.wiki_content.str.contains('{{Character')]

In [93]:
only_characters.iloc[:, [0,1,3]].to_csv('clean_vertex.csv', header=False, index=False)


## Delete old vertices from edges

In [31]:
edge = pd.read_csv('edge.csv')

In [32]:
edge = edge[edge.Target.isin(only_characters.iloc[:, 0].values) & edge.Source.isin(only_characters.iloc[:, 0].values)]

In [33]:
edge.to_csv('clean_edge.csv', index=False)

## Add some attributes to nodes

In [66]:
print(only_characters.wiki_content[43])

{{Character
| Title= Haro
| Image= 
| Season= [[Histories & Lore]]
| Mentioned= "[[The Dothraki]]"
| First=
| Last=
| Aka=
| Status= [[:Category: Deceased individuals|Deceased]]
| Titles= ''[[Khal]]''
| Place= [[Dothraki Sea]]
| Allegiance= Himself
| Father = 
| Actor= 
| Religion = [[Great Stallion]]
| Death =Cut down by the [[Sarnori]] whilst being trampled by their [[horses]] during the [[Field of Crows]] 
| Culture = [[Dothraki]]}}
{{Quote|As battle was joined, the earth-shattering advance of the [[Sarnori|Tall Men]]'s chariots smashed through the center of the [[Dothraki]] horde, the spinning blades on their wheels slicing through the legs of the Dothraki [[horses]]. When one [[khal]] went down before them, cut to pieces and trampled, his ''[[khalasar]]'' broke and fled.|[[Jorah Mormont]]|The Dothraki}}
Khal '''Haro''' was a [[Dothraki]] [[Khal]].

==History==
Khal Haro was one of four ''[[khal]]s'' who fought in what later became known as the [[Field of Crows]]. He was killed by 

In [91]:
only_characters['allegiance'] = only_characters.wiki_content.str.extract(r'Allegiance(.*)\[\[(.*)\]\]\r')[1]

In [48]:
not_redirected[~not_redirected.wiki_content.str.contains('{{Character')]

,Unnamed: 0,wiki_title,wiki_content
3,4,Valryon,{{Animal\r\n|Title = Valryon\r\n|Season = [[Se...
10,12,Jesse Magee,{{Cast\r\n|Title = Jesse Magee\r\n|role = [[Li...
14,17,Eunuchs,[[File:Gelding-knife.jpg|thumb|right|250px|A g...
28,32,Gabrielle Dempsey,{{stub}}{{Template:Cast\r\n|Title = Gabrielle ...
33,41,House Strong,{{Noble house\r\n|Title = House Strong\r\n|Ima...
...,...,...,...
1394,1981,House Martell,{{Noble house\r\n|Title = House Martell\r\n|Im...
1417,2008,Differences in the status of women between boo...,::''This is a sub-page branching off from the ...
1418,2011,Dosh khaleen,{{DISPLAYTITLE:''Dosh khaleen''}}\r\n{{Infobox...
1425,2021,Snow bears,{{Species|Title = Snow bear|Image =Snow bear.p...
